In [1]:
!pip install langchain
!pip install langchain_community


In [2]:
! pip install pypdf

In [8]:
! pip install openai

In [9]:
! pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.8 MB/s eta 0:00:00


In [13]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00


In [16]:
import os

from google.colab import userdata
from langchain.vectorstores import Pinecone
from langchain_community.document_loaders import PyPDFLoader
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter

#openai_api_key = os.getenv("OPENAI_API_KEY")
#pinecone_api_key=os.getenv("pinecone_api_key")
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')
pinecone_index_name = "flowise2"

loader = PyPDFLoader("https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/33ab9a1c-e116-4117-9323-2355cba0fa2f.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=50
    )

texts = text_splitter.split_documents(documents)

vectorstore = Pinecone.from_documents(
    documents=texts,
    embedding= OpenAIEmbeddings(model = "text-embedding-ada-002"),
    index_name = pinecone_index_name
    )

retriever = vectorstore.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=retriever
)

query = "On what date was the document signed ?"
result = qa.run(query)
print(result)

 September 10, 2024
